In [1]:
import pandas

In [4]:
data=pandas.read_csv("review30000.csv")
review_data=pandas.DataFrame()
review_data["reviewbody"]=data["b.reviewbody"]
review_data["口味评分"]=data["口味评分"]
review_data["环境评分"]=data["环境评分"]
review_data["服务评分"]=data["服务评分"]
#把截取的维度数据进行去空
review_data=review_data.dropna(axis=0,how="any")
review_data

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,reviewbody,口味评分,环境评分,服务评分
0,这种地方，环境好是肯定的～吃的羊排什么的。还行吧～,3.0,4.0,4.0
1,这是一家在中国的中餐馆，口味、菜单、摆盘（菜量）和任何一家四星本帮菜都没法比… …,0.0,1.0,1.0
2,天天中午发愁吃什么，刚好走进中信泰富，为了不过多纠结就和小伙伴决定吃意面。意面分量足够，吃完...,3.0,4.0,4.0
3,很多年前在电视上见过介绍这家饭店，就在网上搜索过评论不怎么样也就作罢了，此次有亲戚介绍推荐上...,2.0,0.0,0.0
4,去了好几次，德国猪肘是这里的特色菜，基本上每桌必点！还有这里的生啤也不错！如果要坐外面需要自...,4.0,4.0,4.0
...,...,...,...,...
29995,期待的快餐，菜品丰富，份量足！,4.0,4.0,4.0
29996,强烈推荐崇明中兴镇高大上休闲场所。带上你们的亲朋好友一起来happy！来了会有意外惊喜哟！,4.0,4.0,4.0
29997,水果船味道不错，甜甜的，比较好吃。军舰拼盘实惠，口味多，都市我喜欢的。店内招牌更好说一下了风...,4.0,4.0,4.0
29998,长风景畔广场自从变身为长风大悦城之后，格局大改造，好久没来大变样了，一楼的好多商铺都移至地下...,2.0,1.0,1.0


In [14]:
import jieba
import numpy
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dropout,Dense,Activation
def cut_words(review):
    review["review_cut_words"]=review["reviewbody"].apply(lambda x:list(jieba.cut(x)))
    print("分词化处理结束")
    return review
def clean_stop_words(review):
    with open("stopwords.txt","r",encoding="utf8") as f:
        #希望读出的结果形成一个列表
        stops=f.readlines()
    stops=[x.strip() for x in stops]
    review["review_cut_words"]=review["review_cut_words"].apply(lambda x:[i for i in x if i not in stops])
    print("去掉停用词结束")
    return review
def count_words(review):
    #把分词形成一个新的词的集合
    w=[]
    for i in review["review_cut_words"]:
        w.extend(i)
    review_dict=pandas.DataFrame(pandas.Series(w).value_counts(),columns=["counts"])
    print("词频计算结束")
    return review_dict
def array_words(review,review_dict):
    review["sent"]=review["review_cut_words"].apply(lambda x:list(review_dict["counts"][x]))
    maxlen=50
    review["sent"]=list(sequence.pad_sequences(review["sent"],maxlen))
    print("词向量化计算结束")
    return review
def markdown(review):
    review["totalScore"]=review["口味评分"]+review["环境评分"]+review["服务评分"]
    review["pos_or_neg"]=pandas.Series()
    pos=review[review["totalScore"]>11]
    pos["pos_or_neg"]=1
    neg=review[review["totalScore"]<4]
    neg["pos_or_neg"]=0
    review=pos.append(neg)
    print("标注结束")
    return review
a=review_data[:25000]
a1=cut_words(a)
a2=clean_stop_words(a1)
review_dict_a=count_words(a2)
a3=array_words(a2,review_dict_a)
a4=markdown(a3)
traindata=a4
x=numpy.array(list(traindata["sent"]),dtype=object)[::2]
y=numpy.array(list(traindata["pos_or_neg"]),dtype=object)[::2]
xt=numpy.array(list(traindata["sent"]),dtype=object)[1::2]
yt=numpy.array(list(traindata["pos_or_neg"]),dtype=object)[1::2]
xa=numpy.array(list(traindata["sent"]),dtype=object)
ya=numpy.array(list(traindata["pos_or_neg"]),dtype=object)
model=Sequential()
model.add(Embedding(len(review_dict_a)+1,256))
model.add(LSTM(output_dim=32,activation="sigmoid",inner_activation="hard_sigmoid"))
model.add(Dropout(0.5))
model.add(Dense(input_dim=32,output_dim=1))
model.add(Activation("sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="Adam",metrics=["accuracy"])
model.fit(x,y,batch_size=32,nb_epoch=5,verbose=1,validation_data=(xt,yt),validation_steps=None)
classes=model.predict_classes(xa,batch_size=32,verbose=1)
print("classes=",classes)
score=model.evaluate(xt,yt,verbose=1)
print("score=",score)
b=review_data[25000:26000]
b1=cut_words(b)
b2=clean_stop_words(b1)
review_dict_b=count_words(b2)
b3=array_words(b2,review_dict_b)
b4=markdown(b3)
predict_data=b4
xb=numpy.array(list(predict_data["sent"]),dtype=object)
predict_result=model.predict(xb,batch_size=32,verbose=1)
print("predict_result",predict_result)
predict_classes=model.predict_classes(xb,batch_size=32,verbose=1)
print("predict_classes",predict_classes)

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":


分词化处理结束


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


去掉停用词结束
词频计算结束


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

分词化处理结束


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


去掉停用词结束
词频计算结束
词向量化计算结束
标注结束


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

NameError: name 'objecty' is not defined